In [ ]:
import pandas as pd
import numpy as np
import folium
import matplotlib.pyplot as plt
import math
from sklearn.cluster import DBSCAN
from mlxtend.frequent_patterns import apriori, association_rules 
import warnings
warnings.filterwarnings("ignore")
from sklearn.neighbors import DistanceMetric
from math import radians
import osmium as osm

In [ ]:
# import osmium as osm
# import pandas as pd

# class OSMHandler(osm.SimpleHandler):
#     def __init__(self):
#         osm.SimpleHandler.__init__(self)
#         self.osm_data = []

#     def tag_inventory(self, elem, elem_type):
#         if elem_type == 'node':
#             for tag in elem.tags:
#                 self.osm_data.append([elem_type, 
#                                        elem.id, 
#                                        elem.version,
#                                        elem.visible,
#                                        pd.Timestamp(elem.timestamp),
#                                        elem.uid,
#                                        elem.user,
#                                        elem.changeset,
#                                        len(elem.tags),
#                                        tag.k, 
#                                        tag.v,
#                                      elem.location.lat,
#                                     elem.location.lon]
#                                     )
#         else:
#              for tag in elem.tags:
#                 self.osm_data.append([elem_type, 
#                                        elem.id, 
#                                        elem.version,
#                                        elem.visible,
#                                        pd.Timestamp(elem.timestamp),
#                                        elem.uid,
#                                        elem.user,
#                                        elem.changeset,
#                                        len(elem.tags),
#                                        tag.k, 
#                                        tag.v]
#                                     )

#     def node(self, n):
#         self.tag_inventory(n, "node")

#     def way(self, w):
#         self.tag_inventory(w, "way")

#     def relation(self, r):
#         self.tag_inventory(r, "relation")


# osmhandler = OSMHandler()
# # scan the input file and fills the handler list accordingly
# osmhandler.apply_file("vnroads20201128_q3.osm")

# # transform the list into a pandas DataFrame
# data_colnames = ['type', 'id', 'version', 'visible', 'ts', 'uid',
#                  'user', 'chgset', 'ntags', 'tagkey', 'tagvalue','lat','lon']
# df_osm = pd.DataFrame(osmhandler.osm_data, columns=data_colnames)


In [ ]:
%%time

class OSMHandler(osm.SimpleHandler):
    def __init__(self):
        osm.SimpleHandler.__init__(self)
        self.osm_data = []

    def tag_inventory(self, elem, elem_type):
        if elem_type == 'node':
            dict_value_node = dict()
            dict_value_node['type'] = elem_type
            dict_value_node['lat'] = elem.location.lat
            dict_value_node['lon'] = elem.location.lon
            dict_value_node['id'] = elem.id
            dict_value_node['version'] = elem.version
            dict_value_node['visible'] = elem.visible
            dict_value_node['uid'] = elem.uid
            dict_value_node['user'] = elem.user
            dict_value_node['changeset'] = elem.changeset
            dict_value_node['ntags'] = len(elem.tags)
            for tag in elem.tags:
                dict_value_node[tag.k] = tag.v
            self.osm_data.append(dict_value_node)
        elif elem_type == 'way':
            dict_value_way = dict()
            dict_value_way['type'] = elem_type
            dict_value_way['id'] = elem.id
            dict_value_way['version'] = elem.version
            dict_value_way['visible'] = elem.visible
            dict_value_way['uid'] = elem.uid
            dict_value_way['user'] = elem.user
            dict_value_way['changeset'] = elem.changeset
            dict_value_way['ntags'] = len(elem.tags)
            for tag in elem.tags:
                dict_value_way[tag.k] = tag.v
            self.osm_data.append(dict_value_way)
        else:
            dict_value_relation = dict()
            dict_value_relation['type'] = elem_type
            dict_value_relation['id'] = elem.id
            dict_value_relation['version'] = elem.version
            dict_value_relation['visible'] = elem.visible
            dict_value_relation['uid'] = elem.uid
            dict_value_relation['user'] = elem.user
            dict_value_relation['changeset'] = elem.changeset
            dict_value_relation['ntags'] = len(elem.tags)
            for tag in elem.tags:
                dict_value_relation[tag.k] = tag.v
            self.osm_data.append(dict_value_relation)
        
    def node(self, n):
        self.tag_inventory(n, "node")

    def way(self, w):
        self.tag_inventory(w, "way")

    def relation(self, r):
        self.tag_inventory(r, "relation")


osmhandler = OSMHandler()
# scan the input file and fills the handler list accordingly
osmhandler.apply_file("vnroads20201128_q3.osm")

# transform the list into a pandas DataFrame
df_osm = pd.DataFrame(osmhandler.osm_data)

Wall time: 1.14 s


In [ ]:
df_osm

In [ ]:
df_osm[df_osm['type']=='node']

In [ ]:
df_osm_way = df_osm[df_osm['type']=='way']

In [ ]:
2617602
df_osm_way[df_osm_way['id'] == 2617689]['vbd:namebase']

25237    88 Đường Vườn Chuối
Name: vbd:namebase, dtype: object

In [ ]:
df_osm['type'].unique()

array(['node', 'way', 'restriction'], dtype=object)

In [ ]:
df_osm_restriction = df_osm[df_osm['type']=='restriction']

In [ ]:
df_osm_restriction

,type,lat,lon,id,version,visible,uid,user,changeset,ntags,...,truck:forward,junction,bridge,bus,car:forward,motor,car,motor:forward,restriction,except
30905,restriction,NaN,NaN,757,1,True,1,vietbando,1,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no_left_turn,motor;car;truck
30906,restriction,NaN,NaN,758,1,True,1,vietbando,1,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no_left_turn,motor;car;truck
30907,restriction,NaN,NaN,759,1,True,1,vietbando,1,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no_left_turn,motor;car;truck
30908,restriction,NaN,NaN,761,1,True,1,vietbando,1,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no_left_turn,motor;car;truck
30909,restriction,NaN,NaN,812,1,True,1,vietbando,1,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no_right_turn,motor;car;truck
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30983,restriction,NaN,NaN,2760,1,True,1,vietbando,1,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no_u_turn,motor
30984,restriction,NaN,NaN,2761,1,True,1,vietbando,1,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no_u_turn,motor
30985,restriction,NaN,NaN,2762,1,True,1,vietbando,1,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no_u_turn,motor
30986,restriction,NaN,NaN,2763,1,True,1,vietbando,1,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no_u_turn,motor


In [ ]:
df_osm.isna().sum()

type                  0
lat               10971
lon               10971
id                    0
version               0
visible               0
uid                   0
user                  0
changeset             0
ntags                 0
vbd:layer             0
vbd:namebase      20100
vbd:type          20100
vbd:vtunnel       20100
name              20100
vbd:voverpass     20100
vbd:level         20100
highway           20100
oneway            29788
truck             30879
car:backward      30794
truck:backward    30758
truck:forward     30650
junction          30938
bridge            30971
bus               30987
car:forward       30833
motor             30983
car               30984
motor:forward     30986
restriction       30905
except            30909
dtype: int64

In [ ]:
df_osm['car:backward'].unique()

array([nan, 'no'], dtype=object)

In [ ]:
df_osm['truck:backward'].unique()

array([nan, 'no'], dtype=object)

In [ ]:
df_osm['truck:forward'].unique()

array([nan, 'no'], dtype=object)